In [1]:
import os
import json

import matplotlib.pyplot as plt

from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import numpy as np

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

In [2]:
for file in os.listdir('JSONs_for_TP'):
    converted_data = dict(data=list())
    with open('JSONs_for_TP/' + file, 'r') as f:
        json_data = json.loads(f.read())
        keys = list(json_data.keys())    
        converted_data['keys_'] = keys
        print('keys for {}: {}'.format(file, keys))
        
        for num in json_data[keys[0]].keys():
            data = dict()
            
            for key in keys:
                if key == 'Lat' or key == 'lat':
                    val = json_data[key][num]
                    try:
                        float(val)
                    except:
                        print('file: {}, val for lat: {}'.format(file, val))
                        
                    data['lat'] = json_data[key][num]
                elif key == 'Long' or key == 'lng':
                    val = json_data[key][num]
                    try:
                        float(val)
                    except:
                        print('file: {}, val for lng: {}'.format(file, val))
                    data['lng'] = val
                else:
                    data[key] = json_data[key][num]
                
            converted_data['data'].append(data)
            
        f.close()
        
    with open('d3_jsons/' + file, 'w') as write_file:
        write_file.write(json.dumps(converted_data))
        write_file.close()

keys for earthquakes2.json: ['Year', 'Month', 'Day', 'lat', 'lng', 'Mag', 'Depth']
keys for earthquakes_original.json: ['UTC', 'Country', 'Lat', 'Long', 'Depth_km', 'Mag', 'Secondary_Effects', 'PDE_Shaking_Deaths', 'PDE_Deaths', 'Utsu_Deaths', 'EM-DAT_Deahts', 'Other_Deaths', 'Max_Deaths', 'year', 'month', 'day']
keys for eruptions.json: ['Year', 'Month', 'Day', 'lat', 'lng', 'ExplosivityIndexMax', 'End_Year', 'End_Month', 'End_Day', 'Continuing']
keys for tsunamis.json: ['Year', 'Month', 'Day', 'lat', 'lng', 'Magnitude', 'Focal_Depth', 'Max_Water_Height', 'Deaths', 'Total_Deaths']
file: tsunamis.json, val for lat: None
file: tsunamis.json, val for lng: None
file: tsunamis.json, val for lat: None
file: tsunamis.json, val for lng: None
file: tsunamis.json, val for lat: None
file: tsunamis.json, val for lng: None
file: tsunamis.json, val for lat: None
file: tsunamis.json, val for lng: None
file: tsunamis.json, val for lat: None
file: tsunamis.json, val for lng: None
file: tsunamis.json, 

In [3]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

In [4]:
%%javascript
// console.log(require.s.contexts._.defined);
require(['d3'], function(d3){
    console.log('THIS IS d3: ', d3);
    console.log('d3 version: ', d3.version);
   
});


<IPython.core.display.Javascript object>

In [5]:
%%html
<style>
// add your CSS styling here
h1 {
    font-family: sans-serif;
    font-weight: 300;
    font-size: 16px;
}
</style>

In [6]:
%%javascript
require(['d3'], function(d3) {
    
    var width = 800,
        height = 600;
    
    var hTagsFontFamily = 'sans-serif',
        hTagsFontWeight = '300';
   
    //a weird idempotency thing
    $('#chart1').remove();
    //create canvas
    element.append(`
        <div id='chart1' class='row'>
            <div id='map-title' class='row'><h1>Earthquakes 1900-2011</h1></div>            
            <div id='map-container' class='col-md-8'>
                <div id='map'></div>
            </div>
            <div id='stats-container' class='class='col-md-4'>
                <div class='row'>
                    <h2 id='stats-country'>Country: <span id='stats-country-dynamic'></span></h1>
                    <div id='stats-earthquake-dynamic' class='row'><h2 id='stats-earthquake'>Earthquake: </h2></div>
                </div>                
            </div>
        </div>
    `);
    $('#map-title h1').css({ 'font-family': hTagsFontFamily,  'font-weight': hTagsFontWeight, 'font-size': '40px'});  
    
//     var projection = d3.geo.mercator()
//         .scale(104)
//         .translate([width / 2.45, height / 2]);
      
//     var path = d3.geo.path().projection(projection);
  
    var svg = d3.select('#map').append('svg')
        .attr({
          width: width,
          height: height
        }),
        g = svg.append('g');
    
    // then, create the zoom behvavior
    var zoom = d3.behavior.zoom()
        // only scale up, e.g. between 1x and 50x
        .scaleExtent([1, 50])
        .on("zoom", function() {
          // the "zoom" event populates d3.event with an object that has
          // a "translate" property (a 2-element Array in the form [x, y])
          // and a numeric "scale" property
            
            var e = d3.event,
                // now, constrain the x and y components of the translation by the
                // dimensions of the viewport
                tx = Math.min(0, Math.max(e.translate[0], width - width * e.scale)),
                ty = Math.min(0, Math.max(e.translate[1], height - height * e.scale));

            // then, update the zoom behavior's internal translation, so that
            // it knows how to properly manipulate it on the next movement
            zoom.translate([tx, ty]);
            // and finally, update the <g> element's transform attribute with the
            // correct translation and scale (in reverse order)
            g.attr("transform", [
                "translate(" + [tx, ty] + ")",
                "scale(" + e.scale + ")"
            ].join(" "));
        });
    
    svg
        .call(zoom)
        .call(zoom.event);

    d3.json('custom.geo.json', function(error, json) {
        console.log(json.features);
        
        // Create a unit projection.
        var projection = d3.geo.albers()
            .scale(1)
            .translate([0, 0]);

        // Create a path generator.
        var path = d3.geo.path()
            .projection(projection);

        // Compute the bounds of a feature of interest, then derive scale & translate.
        var b = path.bounds(shapefiles_json),
            s = .95 / Math.max((b[1][0] - b[0][0]) / width, (b[1][1] - b[0][1]) / height),
            t = [(width - s * (b[1][0] + b[0][0])) / 2, (height - s * (b[1][1] + b[0][1])) / 2];        

        // Update the projection to use computed scale & translate.
        projection
            .scale(s)
            .translate(t);
        
        // Draw each province as a path
        svg.selectAll('path')
            .data(json.features)
            .enter().append('path')
                .attr({
                    'd': path,
                    'id': function(d) {return d.properties.name;},
                    'class': function(d) { return ['continent', d.properties.continent]; }
                })
                .style({
                    'fill': '#666',
                    'stroke-opacity': .4,
                    'stroke': '#D3D3D3'
                })
                .on('mouseover', function(d) {
                    d3.select( this )
                        .transition()
                        .style({
                            'stroke-opacity': 1,
                            'stroke': '#f00',
                            'fill': 'pink'
                        });
            
                    $( '#stats-country-dynamic' ).text( d.properties.name ) ;
                })
                .on('mouseout', function(d) {
                    d3.select(this)
                        .transition()
                        .style({
                            'stroke-opacity': .4,
                            'stroke': '#D3D3D3',
                            'fill': '#666'
                        });
            
                    $('#stats-country-dynamic').text( ' ' );
                });
        
        d3.json('data/earthquakes_json.json', function(error, earthquakes_data) {            
            var converted_earthquakes = [];
            var eq_keys = Object.keys(earthquakes_data);
            
            for (var i = 0; i < Object.keys(earthquakes_data['lat']).length; i++) {
                var data_point = new Object();
                
                for (var j = 0; j < eq_keys.length; j++) {
                    var key = eq_keys[j];
                    var value = earthquakes_data[eq_keys[j]][i];
                    if ((key == 'Lat' || key == 'Long')) {
                        if ((value == '?' || value == '??' || value == 0)) {
                            data_point = 0;
                            break;
                        } else {
                            value = parseFloat(value);
                        }

                    }
                    
                    data_point[key] = value;
                }
                if (data_point !== 0) {
                    converted_earthquakes.push(data_point);
                }                
            }
            console.log('converted_earthquakes: ', converted_earthquakes);
            
            svg.selectAll('circle')
                .data(converted_earthquakes).enter()
                .append('circle')
                .attr({
                  cx: function(d) { return projection([d.Long, d.Lat])[0]; },
                  cy: function(d) { return projection([d.Long, d.Lat])[1]; },
                  r: function(d) { 
                      if (d.Magnitude == -1) {
                          return 0;
                      } else {
                          return Math.sqrt(d.Magnitude);;
                      }    
                  },
                  'class': 'earthquake'
                })
                .style('fill', 'red')
                .on('mouseover', function(d) {
                    console.log(d);
                
                    var $earthquakeStats = `
                        <ul>
                            <li><strong>Time of event: ${d.day}/${d.month}/${d.year}</strong></li>
                            <li><strong>Magnitude: ${d.Magnitude}</strong></li>
                            <li><strong>Total Deaths: ${d.deaths}</strong></li>
                            <li><strong>Depth: ${d['Depth (km)']}(km)</strong></li>
                        </ul>
                    `;
                
                    $( '#stats-earthquake-dynamic' ).append( $earthquakeStats );
                })
                .on('mouseout', function(d) {
                    $( '#stats-earthquake-dynamic ul' ).remove();
                });
            });
            
            console.log(svg.selectAll('circle'));
    });
});

<IPython.core.display.Javascript object>

In [7]:
%%html
<style>
svg {
    border: 1px solid #3f87a6;
},

h1 {
    font-family: sans-serif;
    font-weight: lighter;
}
</style>

In [8]:
%%javascript
require(['d3'], function(d3) {
    var data_pieces = {         
        tsunamis: { name: 'tsunamis', file_path: 'd3_jsons/tsunamis.json', color: 'blue', circlePath: '.blue.dot', className: 'blue dot' }, 
        eruptions: { name: 'eruptions', file_path: 'd3_jsons/eruptions.json', color: 'red', circlePath: '.red.dot', className: 'red dot' },
        earthquakes: { name: 'earthquakes', file_path: 'd3_jsons/earthquakes_original.json', color: 'green', circlePath: '.green.dot', className: 'green dot' }, 
    };
    
    $('#chart1').remove();
    element.append(`
        <div id='chart1' class='row'>
            <h1 id='map-title' style="font-family: sans-serif; font-weight: lighter">Earthquakes 1900-2011</h1>
            <div id='map-container container' class='col-md-12'>
                <div id='map'></div>
            </div>
        </div>
    `);
//     <div class='container'>
//                 <div id='stats-container' class='col-md-4'>
//                     <div class='row'>
//                         <h2 id='stats-country'>Country: <span id='stats-country-dynamic'></span></h1>
//                         <div id='stats-earthquake-dynamic' class='row'><h2 id='stats-earthquake'>Earthquake: </h2></div>
//                         <div id='stats-volcanos-dynamic' class='row'><h2 id='stats-earthquake'>Volcano: </h2></div>
//                         <div id='stats-tsunamis-dynamic' class='row'><h2 id='stats-earthquake'>Tsunami: </h2></div>
//                     </div>                
//                 </div>
//             </div>

    var width = 950,
        height = 400;
  
    var svg = d3.select('#map').append('svg')
        .attr({
          width: 650,
          height: height
        }),
        g = svg.append('g');
    
    // then, create the zoom behvavior
    var zoom = d3.behavior.zoom()
        // only scale up, e.g. between 1x and 50x
        .scaleExtent([1, 50])
        .on("zoom", function() {
          // the "zoom" event populates d3.event with an object that has
          // a "translate" property (a 2-element Array in the form [x, y])
          // and a numeric "scale" property
            
            var e = d3.event,
                // now, constrain the x and y components of the translation by the
                // dimensions of the viewport
                tx = Math.min(0, Math.max(e.translate[0], width - width * e.scale)),
                ty = Math.min(0, Math.max(e.translate[1], height - height * e.scale));

            // then, update the zoom behavior's internal translation, so that
            // it knows how to properly manipulate it on the next movement
            zoom.translate([tx, ty]);
            // and finally, update the <g> element's transform attribute with the
            // correct translation and scale (in reverse order)
            g.attr("transform", [
                "translate(" + [tx, ty] + ")",
                "scale(" + e.scale + ")"
            ].join(" "));
        });
    
    svg
        .call(zoom)
        .call(zoom.event);
    
    function createAppendTooltip(d, name) {
        var $earthquakeStats = `
            <ul>
                <li><strong>Time of event: ${d.day}/${d.month}/${d.year}</strong></li>
            </ul>
        `;
        
        var idName = '#stats-' + name + '-dynamic';

        $( idName ).append( $earthquakeStats );
    }

    d3.json('custom.geo.json', function(error, json) {
        
        // Create a unit projection.
        var projection = d3.geo.mercator()
            .scale(1)
            .translate([0, 0]);

        // Create a path generator.
        var path = d3.geo.path()
            .projection(projection);

        // Compute the bounds of a feature of interest, then derive scale & translate.
        var b = path.bounds(json),
            s = 1.25 / Math.max((b[1][0] - b[0][0]) / width, ((b[1][1] - b[0][1]) / height) * 1.2),
            t = [(width - s * (b[1][0] + b[0][0])) * .68 / 2, (height - s * (b[1][1] + b[0][1])) / 2];        

        // Update the projection to use computed scale & translate.
        projection
            .scale(s)
            .translate(t);
        
        // Draw each province as a path
        g.selectAll('path')
            .data(json.features)
            .enter().append('path')
                .attr({
                    'd': path,
                    'id': function(d) {return d.properties.name;},
                    'class': function(d) { return ['continent', d.properties.continent]; }
                })
                .attr("data-legend", function(d) { return d.properties.continent })
                .style({
                    'fill': '#666',
                    'stroke-opacity': .4,
                    'stroke': '#D3D3D3'
                })
                .on('mouseover', function(d) {
                    d3.select( this )
                        .transition()
                        .style({
                            'stroke-opacity': 1,
                            'stroke': '#f00',
                            'fill': 'pink'
                        });
            
                    $( '#stats-country-dynamic' ).text( d.properties.name ) ;
                })
                .on('mouseout', function(d) {
                    d3.select(this)
                        .transition()
                        .style({
                            'stroke-opacity': .4,
                            'stroke': '#D3D3D3',
                            'fill': '#666'
                        });
            
                    $('#stats-country-dynamic').text( ' ' );
                });

        var className, circlePath;
        
        function createCircles(d) {
            return new Promise((resolve, reject) => {
                let data = data_pieces[d];

                console.log('data', data, 'class', data.className, 'path', data.circlePath);

                d3.json(data.file_path, function(error, json_data) {
                    if (data.color == 'green') { console.log(json_data); }

                    g.selectAll(data.circlePath)
                        .data(json_data.data).enter()
                        .append('circle')
                        .attr({
                          cx: function(d) { 
                              if (d.lng && d.lat) { return projection([d.lng, d.lat])[0]; }
                              else { return projection(['0.0', '0.0'])[0]; }
                          },
                          cy: function(d) { 
                              if (d.lng && d.lat) { return projection([d.lng, d.lat])[1]; }
                              else { return projection(['0.0', '0.0'])[1]; }
                          },
                          r: function(d) { 
                              if (!d.lng && !d.lat && data.color == 'green') { console.log('lat', d.lat, 'lng', d.lng); }
                              return (d.lng && d.lat) ? '2': '0'; 
                          },
                          'class': data.className
                        })
                        .style({
                            'fill': data.color,
                            'opacity': .6
                        })
                        .on('mouseover', function(d) {
                            console.log('Mouse over: ', d);
                            if (data.name == 'earthquakes') { createAppendTooltip(d, 'earthquakes'); }
                            else if (data.name == 'tsunamis') { createAppendTooltip(d, 'tsunamis'); }
                            else if (data.name == 'eruptions') { createAppendTooltip(d, 'eruptions'); }
                        
                        })
                        .on('mouseout', function(d) {
                            console.log('Mouse out');
                            if (data.name == 'earthquakes') { $( '#stats-earthquakes-dynamic ul' ).remove(); }
                            else if (data.name == 'tsunamis') { $( '#stats-tsunamis-dynamic ul' ).remove(); }
                            else if (data.name == 'eruptions') { $( '#stats-eruptions-dynamic ul' ).remove(); }                            
                        });
                    
                    resolve();
                    
                });
            });
        }
        
        Promise.all([createCircles('earthquakes'), createCircles('eruptions'), createCircles('tsunamis')]).then(
                resolves => { console.log('done'); },
                reason => { console.log(reason); });
        

    });
});

<IPython.core.display.Javascript object>